# Binary operations in Python

In [9]:
0x01 & 0x01, 0x01 | 0x10

(1, 17)

In [1]:
0x01 ^ 0x01

0

In [3]:
0x01 ^ 0x10

17

In [4]:
f'{0x01 ^ 0x10:08b}'

'00010001'

In [5]:
f'{0x01 ^ 0x10:08x}'

'00000011'

In [2]:
f'{0x10:08b}'

'00010000'

In [1]:
f'{0x10 >> 1:08b}'

'00001000'

In [24]:
# Bitshift
f'{0x10 >> 1:08b}', f'{0x10 >> 2:08b}', f'{0x10 << 2:08b}'

('00001000', '00000100', '01000000')

In [61]:
import secrets
b = secrets.token_bytes(100)

ands = 0x1f
ors = 0x1f
xors = 0x1f

for bb in b:
    ands &= bb
    ors |= bb
    xors ^= bb
    
ands, ors, xors

(0, 255, 86)

In [23]:
#bit rotate
def bin(n):
    return f'{n:08b}'

def rotr(n, rotations, width=8):
    rotations %= width
    return (n >> rotations) | (n << (width - rotations)) & 0xff

def rotl(n, rotations, width=8):
    rotations %= width
    return (n << rotations) & 0xff | (n >> (width - rotations))

bin(0x50), bin(rotr(0x50, 1)), bin(rotr(0x50, 2)), bin(rotr(0x50, 5)), bin(rotl(0x50, 2))

('01010000', '00101000', '00010100', '10000010', '01000001')

# Padding

In [ ]:
# Padding scheme: Zero Padding

def pad(data: bytes, block_size: int, b: bytes = b'\x00') -> bytes:
    if len(data) > 0 and len(data) % block_size == 0:
        # If the data is already a multiple of the block size, we don't need to pad it, except if the data is empty
        return data
    padding = block_size - len(data) % block_size
    return data + b * padding

def unpad(data: bytes, b: bytes = b'\x00') -> bytes:
    return data.rstrip(b)


assert pad(b'hello', 8) == b'hello\x00\x00\x00'
assert pad(b'welcome stranger', 8) == b'welcome stranger'
assert pad(b'welcome stranger!', 8) == b'welcome stranger!\x00\x00\x00\x00\x00\x00\x00'
assert pad(b'', 4) == b'\x00\x00\x00\x00'
assert unpad(b'\x00\x00') == b''
assert unpad(pad(b'hello', 8)) == b'hello'

In [3]:
# Padding scheme: ANSI X.923

def pad(data: bytes, block_size: int) -> bytes | None:
    if block_size < 1 or block_size > 256:
        return None # Invalid block size
    pad_len = block_size - (len(data) % block_size)
    if pad_len == 0:
        pad_len = block_size
    return data + b'\x00' * (pad_len - 1) + bytes([pad_len])

def unpad(data: bytes) -> bytes | None:
    pad_len = data[-1]
    if pad_len < 1 or pad_len > len(data):
        return None # Invalid padding
    return data[:-pad_len]


assert pad(b'hello', 8) == b'hello\x00\x00\x03'
assert pad(b'123', 4) == b'123\x01'
assert pad(b'1234', 4) == b'1234\x00\x00\x00\x04'
assert pad(b'welcome stranger', 8) == b'welcome stranger\x00\x00\x00\x00\x00\x00\x00\x08'
assert pad(b'welcome stranger!', 8) == b'welcome stranger!\x00\x00\x00\x00\x00\x00\x07'
assert pad(b'', 4) == b'\x00\x00\x00\x04'
assert unpad(b'\x00\x00\x00\x02') == b'\x00\x00'
assert unpad(pad(b'hello', 8)) == b'hello'
assert unpad(pad(b'this is a long message, taking up multiple blocks', 4)) == b'this is a long message, taking up multiple blocks'

In [2]:
# Padding scheme: ISO 7816-4

def pad(data: bytes, block_size: int) -> bytes | None:
    if block_size < 1:
        return None # Invalid block size
    pad_len = block_size - (len(data) % block_size)
    if pad_len == 0:
        pad_len = block_size
    return data + b'\x80' + b'\x00' * (pad_len - 1)

def unpad(data: bytes) -> bytes | None:
    # Find the last occurrence of 0x80 and remove it and any trailing zeros.
    index = data.rfind(b'\x80')
    if index == -1:
        return None
    return data[:index]


assert pad(b'hello', 8) == b'hello\x80\x00\x00'
assert pad(b'123', 4) == b'123\x80'
assert pad(b'1234', 4) == b'1234\x80\x00\x00\x00'
assert pad(b'welcome stranger', 8) == b'welcome stranger\x80\x00\x00\x00\x00\x00\x00\x00'
assert pad(b'welcome stranger!', 8) == b'welcome stranger!\x80\x00\x00\x00\x00\x00\x00'
assert pad(b'', 4) == b'\x80\x00\x00\x00'
assert unpad(b'\x00\x00\x80\x00') == b'\x00\x00'
assert unpad(pad(b'hello', 8)) == b'hello'
assert unpad(pad(b'this is a long message, taking up multiple blocks', 4)) == b'this is a long message, taking up multiple blocks'